In [111]:
# loading libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import math
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import missingno as msno

from sklearn.preprocessing import StandardScaler
from scipy import stats

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [112]:
data = pd.read_csv('/kaggle/input/international-football-results-from-1872-to-2017/results.csv')
data.head()

In [113]:
# deleting all data before the first World Cup, hence before 1930
drop_row_index = data[data['date'] < '1930-07-13'].index
data = data.drop(drop_row_index)

In [114]:
data.shape

In [115]:
data.info()

In [116]:
# not really relevant, but looks cool
data.describe()

In [117]:
# same here
data.describe(include=['object', 'bool'])

In [118]:
# checking if data is more or less full
data.isnull().sum()

The data is full!!!

Lets see how many friendly matches there are and remove them, because it is just not interesting:

In [119]:
data['tournament'].value_counts()

In [120]:
# removing friendly matches
drop_row_index = data[data['tournament'] == 'Friendly'].index
data = data.drop(drop_row_index)

Lets remove tournaments that had less than 50 matches:

In [121]:
# getting the count of each value
value_counts = data['tournament'].value_counts()

# selecting the tournaments where less than 50 matches were played
tour = value_counts[value_counts < 100].index

# keeping rows where the tournament column is not in the tour variable
data = data[~data.tournament.isin(tour)]

Now let us visualize the filtered data and see which tournament was played most:

In [122]:
plt.figure(figsize = (15, 10))
sns.countplot(x="tournament", data = data, order = data['tournament'].value_counts().index)
plt.xticks(rotation = 90)

Obviously, the most played tournaments are the FIFA World Cup and the UEFA Champions league, followed by the African Cup of Nations (qualifications) and so on.

I also wanted to know how many games were actually "fair", i.e how many were played on neutral territory (it plays role):

In [123]:
data['neutral'].value_counts()

7351, not bad imho

Now let us take a look at our second dataset – shootouts.csv. I just want to now the country that won most.

In [124]:
data2 = pd.read_csv('/kaggle/input/international-football-results-from-1872-to-2017/shootouts.csv')
data2.head()

In [125]:
data2['winner'].value_counts()